<a href="https://colab.research.google.com/github/abs711/abs711.github.io/blob/main/Finding_the_submarine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import drive

# the following command loads up your google drive. It will promp you to give Colab access to your Gdrive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import numpy as np

data_path = '/content/drive/My Drive/AMATH 582/Ass1/subdata.npy'

d = np.load(data_path) # huge matrix of size 262144 x 49 (columns contain flattened 3d matrix of size 64x64x64)

In [54]:
d.shape


(262144, 49)

In [55]:
# import libraries for plotting isosurfaces
import plotly
import plotly.graph_objs as go
# utility for clearing output of cell as loop runs in notebook
from IPython.display import clear_output
# Other plotting utilities
import imageio
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from sys import argv

In [56]:
!pip install kaleido


In [ ]:

# plot the original data in time 

L = 10; # length of spatial domain (cube of side L = 2*10)
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # frequency grid for one coordinate

xv, yv, zv = np.meshgrid( x, y, z) # generate 3D meshgrid for plotting

# plot iso surfaces for every third measurement



    
for j in range(0,1,1):

  signal = np.reshape(d[:, j], (N_grid, N_grid, N_grid))
  normal_sig_abs = np.abs(signal)/np.abs(signal).max()

  # generate data for isosurface of the 3D data 
  fig_data = go.Isosurface( x = xv.flatten(), y = yv.flatten(), z = zv.flatten(),
                           value = normal_sig_abs.flatten(), isomin=0.7, isomax=0.7)

# generate plots
  #clear_output(wait=True) # need this to discard previous figs
  fig = go.Figure( data = fig_data )
  fig.show()


In [58]:
signalFT = 0
for j in range(0,49):
  signal = np.reshape(d[:, j], (N_grid, N_grid, N_grid))
  signalFT = signalFT + np.fft.fftshift(np.fft.fftn(signal))

signalFT = signalFT/49




In [ ]:
signalFT_abs = np.abs(signalFT)/np.abs(signalFT).max()
fig_data = go.Isosurface( x = xv.flatten(), y = yv.flatten(), z = zv.flatten(),
                           value = signalFT_abs.flatten(), isomin=0.7, isomax=0.7)

# generate plots
  #clear_output(wait=True) # need this to discard previous figs
fig = go.Figure( data = fig_data)
fig.update_layout(title="Isosurface (isovalue = 0.7) for signal averaged in the Fourier Domain ")
fig.show()

In [ ]:

k1, k2 = np.meshgrid(K_grid, K_grid)

from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = go.Figure()
for i in range(0,64):
  fig.add_trace(go.Surface(z=np.abs(signalFT[:,:,i]), x=K_grid, y=K_grid,cmin=0,cmax=100))


fig.update_layout(title="Overlayed surface plots of abs(averaged Fourier Signal) vs (x, y) coordinates, while iterating over z coordinate values.",width=1000, height=1000,scene = dict(xaxis_title= "k_x",yaxis_title= "k_y",zaxis_title="abs(mean Fourier signal)"))
fig.show()

In [ ]:
fig1 = go.Figure()
for i in range(0,64):
  fig1.add_trace(go.Surface(z=np.abs(signalFT[:,i,:]), x=K_grid, y=K_grid,cmin=0,cmax=100))

fig1.update_layout(title="Overlayed surface plots of abs(averaged Fourier Signal) vs (x, z) coordinates, while iterating over y coordinate values.",scene = dict(yaxis_title= "z", zaxis_title="abs(mean Fourier signal)"))
fig1.show()

In [ ]:
fig2 = go.Figure()
for i in range(0,64):
  fig2.add_trace(go.Surface(z=np.abs(signalFT[i,:,:]), x=K_grid, y=K_grid,cmin=0,cmax=100))

fig2.update_layout(title="Overlayed surface plots of abs(averaged Fourier Signal) vs (y, z) coordinates, while iterating over x coordinate values.",scene = dict(xaxis_title= "y", yaxis_title= "z", zaxis_title="abs(mean Fourier signal)"))
fig2.show()

In [13]:
def g(x,y,z, s): # 2D Gaussian filter
  val = np.exp( - ( (x+5.341)**2 + (y+2.199)**2 + (z-6.911)**2)/(2*s**2) ) + np.exp( - ( (x-5.341)**2 + (y-2.199)**2 + (z+6.911)**2)/(2*s**2)  )

  return val

In [14]:
k11, k22, k33 = np.meshgrid(K_grid, K_grid, K_grid)
sigma = 1 # kill stuff outside k = -100 and 100
g_vals = g(k11, k22, k33, sigma)

g_vals.shape

(64, 64, 64)

In [ ]:
fig3 = go.Figure()
for i in range(0,64):
  fig3.add_trace(go.Surface(z=g_vals[:,:,i], x=K_grid, y=K_grid,cmin=0,cmax=1))


fig3.update_layout(title="Overlayed surface plots of the Filter in Fourier domain vs (x, y) coordinates, while iterating over x coordinate values.",width=1000, height=1000,scene = dict(xaxis_title= "k_x", yaxis_title= "k_y", zaxis_title="Filter"))
fig3.show()

In [65]:
filt_signalFT = g_vals*signalFT

In [ ]:
fig4 = go.Figure()
for i in range(0,64):
  fig4.add_trace(go.Surface(z=np.abs(filt_signalFT[:,:,i]), x=K_grid, y=K_grid,cmin=0,cmax=100))

fig4.update_layout(title="Overlayed surface plots of abs(Filtered average Fourier Signal) vs (x, y) coordinates, while iterating over x coordinate values.",width=1000, height=1000,scene = dict(xaxis_title= "k_x", yaxis_title= "k_y", zaxis_title="abs(Filtered mean Fourier signal)"))
fig4.show()

In [ ]:
L = 10; # length of spatial domain (cube of side L = 2*10)
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # frequency grid for one coordinate

xv, yv, zv = np.meshgrid( x, y, z) # generate 3D meshgrid for plotting

coords = []
denoised_signal_abs = []
for j in range(0,49,1):

  signal = np.reshape(d[:, j], (N_grid, N_grid, N_grid))
  signalFT = np.fft.fftshift(np.fft.fftn(signal))
  filt_signalFT = g_vals*signalFT
  filt_signal = np.real(np.fft.ifftn( np.fft.ifftshift(filt_signalFT) ))
  normal_filtsig_abs = np.abs(filt_signal)/np.abs(filt_signal).max()
  coords.append(np.where(normal_filtsig_abs == np.max(normal_filtsig_abs)))
  denoised_signal_abs.append(normal_filtsig_abs) 
  # generate data for isosurface of the 3D data 
  fig_data = go.Isosurface( x = xv.flatten(), y = yv.flatten(), z = zv.flatten(),
                           value = normal_filtsig_abs.flatten(), isomin=0.7, isomax=0.7)

# generate plots
  clear_output(wait=True) # need this to discard previous figs
  fig = go.Figure( data = fig_data )
  fig.show()



In [26]:
denoised_signal_abs = np.array(denoised_signal_abs)

denoised_signal_abs.shape

(49, 64, 64, 64)

In [27]:
coords_array = np.array(coords)

coords_array.shape


(49, 3, 1)

In [49]:
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]


fig = go.Figure(data=[go.Scatter3d(x=x[np.reshape(coords_array[:,0],[49])],
    y=x[np.reshape(coords_array[:,1],[49])],
    z=x[np.reshape(coords_array[:,2],[49])])])

# tight layout
fig.update_layout(scene = dict(xaxis_title= "x", yaxis_title= "y", zaxis_title="z", xaxis = dict(nticks=10, range=[-10,10],),
                     yaxis = dict(nticks=10, range=[-10,10],),
                     zaxis = dict(nticks=10, range=[-10,10],)),width=800, height=800)

fig.show()

In [44]:
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]


fig = go.Figure(data=[go.Scatter(mode='markers',x=x[np.reshape(coords_array[:,0],[49])],
    y=x[np.reshape(coords_array[:,1],[49])])])


# tight layout
fig.update_layout(xaxis_title= "x", yaxis_title= "y", xaxis = dict(nticks=20, range=[-10,10],),
                     yaxis = dict(nticks=40, range=[-10,10],),width=800, height=400)

fig.show()

In [70]:
fig = go.Figure(data=[go.Scatter(mode='markers',x = np.reshape(t,[49]),
    y=x[np.reshape(coords_array[:,0],[49])],name="x position")])

fig.add_trace(go.Scatter(mode='markers',x = np.reshape(t,[49]),
    y=x[np.reshape(coords_array[:,1],[49])], name="y position"))

fig.add_trace(go.Scatter(mode='markers',x = np.reshape(t,[49]),
    y=x[np.reshape(coords_array[:,2],[49])],name="z position"))

# tight layout
fig.update_layout(xaxis_title= "time (hours)", yaxis_title= "position", xaxis = dict(nticks=20, range=[-0.5,24.5],),
                     yaxis = dict(nticks=20, range=[-10,10],),width=800, height=400)

fig.show()